In [1]:
# Load relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
import warnings
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

In [5]:
## load data set
carmax_df = pd.read_csv('CaseCompetitionData2021.csv')
top10_df = pd.read_csv('Top10ByIncome.csv')

## Time to Cluster - Gonna use k-means initially 




In general, we can cluster observations on the basis of the features in order to identify subgroups among the observations, or we can cluster fea- tures on the basis of the observations in order to discover subgroups among the features. In what follows, for simplicity we will discuss clustering obser- vations on the basis of the features, though the converse can be performed by simply transposing the data matrix.

In [12]:
top10_df.columns
top10_df = top10_df.drop('Unnamed: 0', axis=1)

In [34]:
top10_df['purchase_model']

0        CAMRY
1       ALTIMA
2      COROLLA
3       MALIBU
4       ACCORD
        ...   
115     ACCORD
116    COROLLA
117      CIVIC
118       CR-V
119      FOCUS
Name: purchase_model, Length: 120, dtype: object

In [91]:
cluster_df = carmax_df[carmax_df['purchase_model'].isin(list(set(top10_df['purchase_model'])))]
cluster_df = cluster_df.drop('insert_num', axis=1)

In [92]:
cluster_df

,purchase_make,purchase_model,purchase_vehicle_year,purchase_price,trade_in,vehicle_financing,customer_age,customer_income,customer_gender,customer_previous_purchase,customer_distance_to_dealer,post_purchase_satisfaction,vehicle_warranty_used,subsequent_purchases
1,FORD,F150,2007,15001 - 20000,0,0,51 - 60,0 - 20000,F,1,19,?,0,0
2,BMW,328,2010,25001 - 30000,1,1,41 - 50,60001 - 80000,F,1,21,?,0,0
5,NISSAN,ALTIMA,2009,15001 - 20000,1,1,21 - 30,60001 - 80000,U,0,2,?,1,1
8,FORD,ESCAPE,2010,15001 - 20000,1,1,41 - 50,40001 - 60000,U,1,14,?,1,0
10,NISSAN,SENTRA,2009,10001 - 15000,0,1,61 - 70,20001 - 40000,U,0,?,?,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355854,FORD,FOCUS,2013,20001 - 25000,1,1,31 - 40,100001 - 120000,M,1,4,?,0,2
355860,HYUNDAI,SONATA,2011,20001 - 25000,1,0,41 - 50,?,M,0,7,?,0,1
355864,HONDA,ACCORD,2007,15001 - 20000,0,1,21 - 30,20001 - 40000,F,0,12,?,0,0
355866,TOYOTA,RAV4,2011,20001 - 25000,1,1,51 - 60,40001 - 60000,F,1,3,?,0,0


In [93]:
len(list(set(top10_df['purchase_model'])))

30

In [94]:
cluster_df.dtypes

purchase_make                  object
purchase_model                 object
purchase_vehicle_year           int64
purchase_price                 object
trade_in                        int64
vehicle_financing               int64
customer_age                   object
customer_income                object
customer_gender                object
customer_previous_purchase      int64
customer_distance_to_dealer    object
post_purchase_satisfaction     object
vehicle_warranty_used           int64
subsequent_purchases            int64
dtype: object

In [95]:
for i in cluster_df.columns:
    if cluster_df[i].dtype == 'object':
        classes = cluster_df[i]
        classnames, factorval = np.unique(classes, return_inverse = True)
        cluster_df[i] = factorval

In [96]:
cluster_df.dtypes

purchase_make                  int64
purchase_model                 int64
purchase_vehicle_year          int64
purchase_price                 int64
trade_in                       int64
vehicle_financing              int64
customer_age                   int64
customer_income                int64
customer_gender                int64
customer_previous_purchase     int64
customer_distance_to_dealer    int64
post_purchase_satisfaction     int64
vehicle_warranty_used          int64
subsequent_purchases           int64
dtype: object

In [97]:
cluster_df

,purchase_make,purchase_model,purchase_vehicle_year,purchase_price,trade_in,vehicle_financing,customer_age,customer_income,customer_gender,customer_previous_purchase,customer_distance_to_dealer,post_purchase_satisfaction,vehicle_warranty_used,subsequent_purchases
1,3,12,2007,1,0,0,5,0,0,1,116,10,0,0
2,0,0,2010,3,1,1,4,9,0,1,135,10,0,0
5,9,2,2009,1,1,1,2,9,2,0,127,10,1,1
8,3,11,2010,1,1,1,4,8,2,1,46,10,1,0
10,9,23,2009,0,0,1,6,7,2,0,320,10,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355854,3,13,2013,2,1,1,3,1,1,1,231,10,0,2
355860,5,25,2011,2,1,0,4,11,1,0,286,10,0,1
355864,4,1,2007,1,0,1,2,7,0,0,24,10,0,0
355866,10,21,2011,2,1,1,5,8,0,1,212,10,0,0


In [98]:
from sklearn.preprocessing import StandardScaler

X = cluster_df.values[:,1:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

array([[-0.0700046 , -1.30434939, -0.18339136, ...,  0.05222532,
        -0.5351671 , -0.44335392],
       [-1.4758237 ,  0.0304836 ,  1.12888161, ...,  0.05222532,
        -0.5351671 , -0.44335392],
       [-1.24152051, -0.41446073, -0.18339136, ...,  0.05222532,
         1.86857526,  0.39398434],
       ...,
       [-1.3586721 , -1.30434939, -0.18339136, ...,  0.05222532,
        -0.5351671 , -0.44335392],
       [ 0.98435973,  0.47542793,  0.47274513, ...,  0.05222532,
        -0.5351671 , -0.44335392],
       [ 0.51575336,  0.92037227, -0.18339136, ...,  0.05222532,
        -0.5351671 , -0.44335392]])

Lets run model cluster

In [99]:
num_clusters = 5

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[2 1 0 ... 3 1 1]


In [100]:
cluster_df["Labels"] = labels
cluster_df.head(5)

,purchase_make,purchase_model,purchase_vehicle_year,purchase_price,trade_in,vehicle_financing,customer_age,customer_income,customer_gender,customer_previous_purchase,customer_distance_to_dealer,post_purchase_satisfaction,vehicle_warranty_used,subsequent_purchases,Labels
1,3,12,2007,1,0,0,5,0,0,1,116,10,0,0,2
2,0,0,2010,3,1,1,4,9,0,1,135,10,0,0,1
5,9,2,2009,1,1,1,2,9,2,0,127,10,1,1,0
8,3,11,2010,1,1,1,4,8,2,1,46,10,1,0,0
10,9,23,2009,0,0,1,6,7,2,0,320,10,0,1,3


In [101]:
cluster_df.groupby('Labels').mean()

,purchase_make,purchase_model,purchase_vehicle_year,purchase_price,trade_in,vehicle_financing,customer_age,customer_income,customer_gender,customer_previous_purchase,customer_distance_to_dealer,post_purchase_satisfaction,vehicle_warranty_used,subsequent_purchases
Labels,,,,,,,,,,,,,,
0,5.435972,12.227309,2009.242654,1.178553,0.457520,0.816404,3.408619,7.738446,0.711456,0.174128,180.685228,9.998404,0.999964,0.557680
1,5.647114,12.794962,2010.274222,1.419870,0.999948,0.789698,3.558343,8.449225,0.684194,0.233895,181.605635,9.998298,0.000000,0.543469
2,5.657454,13.169120,2010.120833,1.582500,0.487824,0.818333,4.403472,0.848472,0.706019,0.323287,180.529954,9.997315,0.140694,0.514028
3,5.531020,12.403428,2009.967015,1.098597,0.000000,0.765191,3.381518,8.371401,0.724762,0.122603,183.772394,9.998424,0.000000,0.509888
4,5.775701,12.753894,2010.024922,1.102804,0.454829,0.781931,3.638629,7.124611,0.685358,0.283489,175.697819,2.024922,0.227414,0.542056
